# Librerias

In [ ]:
import PySimpleGUI as sg
import os
import pandas as pd
import shutil

# Organizador de Archivos

In [ ]:
def organizador(values):
    
    #---------ENTRADA---------
    # Lista de archivos y clasificación csv
    master_files = values['Indice']

    # Lista de carpetas a crear e identificador
    master_direct = values['Codigo_Nat']

    # Carpeta donde se encuentran todos los archivos
    source = values['Carpeta_Entrada']
    
    #----------SALIDA----------
    # Carpeta donde se crearan las carpetas de salida
    out_direct = values['Carpeta_Salida']
    
    #--------DataFrames--------
    # Crear dataframe de master_files
    df_master = pd.read_csv(master_files, encoding='utf_8_sig')

    # Crear dataframe usando master_direct
    df_master_dir = pd.read_csv(master_direct, encoding='utf_8_sig')
    
    #------Verificar que todo OK---------
    # Crear lista con archivos de master_files
    files_in_master = df_master['Nombre Archivo'].tolist()
    # Cuantos archivos hay en el indice
    nFiles_Indice = len(files_in_master) 

    # Crear lista con ID de archivos en master_files
    id_master = df_master['No.'].tolist()

    # Leer archivos de la carpeta concentradora
    files_in_source = os.listdir(source)


    # Verificar que todos los archivos de master_files existan en la concentradora
    # Archivos faltantes en carpeta concentradora

    miss_files = []
    miss_id = []

    for i in range(len(files_in_master)):
        if files_in_master[i] in files_in_source:
            continue
        else:
            miss_files.append(files_in_master[i])
            miss_id.append(id_master[i])



    if not miss_files:
        #--------Modificar DataFrames---------
        # Agregar primera letra de folder destino para match con diccionario
        df_master['Naturaleza'] = df_master['Código'].astype(str).str[0]

        # Crear diccionario de carpetas usando master_direct
        diccionario = pd.Series(df_master_dir.Naturaleza.values, index=df_master_dir.Codigo).to_dict()

        # Sustituir primera letra por nombre completo
        df_master = df_master.replace({"Naturaleza": diccionario})

        #------Verificar Num. Carpetas--------
        # Crear lista con carpetas a crear
        folder_list = df_master_dir['Naturaleza'].tolist()
        # Misma cantidad de carpetas que de clases?
        if len(df_master.Naturaleza.unique()) == len(folder_list):
            #--------Copiar archivos a nuevas carpetas------
            for index, row in df_master.iterrows():
                # De donde sale el archivo
                filename = row['Nombre Archivo']
                source_file = os.path.join(source, filename)

                # A donde va el archivo
                folder_1 = str(row['Naturaleza'])
                folder_2 = str(row['Año'])
                folder_3 = str(row['Autor/Fuente'])
                folder_destino = os.path.join(out_direct, folder_1, folder_2, folder_3)

                # Si la ruta de destino no existe, crearla
                os.makedirs(folder_destino, exist_ok=True)
                # Ruta para con archivo incluido
                file_dest = os.path.join(folder_destino, filename)

                # Copiar archivo a nueva carpeta
                try:
                    shutil.copyfile(source_file, file_dest)

                  # Otros errores
                except:
                    break
    else:
        df_missing = pd.DataFrame(list(zip(miss_id, miss_files)), columns=['No.', 'Nombre'])
        out1 = values['Carpeta_Salida']
        out_file = os.path.join(out1, errores.csv)
        df_missing.to_csv(out_file, index=False, encoding='utf_8_sig')
    
    
    

# Interfaz
sg.theme_previewer()

values    
0 -> carpeta contenedora de archivos     
0 -> carpeta de salida

In [ ]:
sg.theme('Material1')


layout = [
    [sg.Text('Sección de carpetas')],
    [sg.Text(), ],
    [sg.Text('Seleccione la carpeta donde estan todos los archivos', size=(40,1)), sg.InputText(size=(30,1), key='Carpeta_Entrada'), sg.FolderBrowse('Buscar')],
    [sg.Text('Seleccione carpeta principal donde se crearán el resto de sub-carpetas', size=(40,2)), sg.InputText(size=(30,1), key='Carpeta_Salida'), sg.FolderBrowse('Buscar')],
    [sg.Text('Sección para cargar archivos')],
    [sg.Text()],
    [sg.Text('Seleccione el Indice (Excel)', size=(40,1)), sg.Input(size=(30,1), key='Indice'), sg.FileBrowse('Buscar')],
    [sg.Text('Seleccione el codigo de naturaleza (Excel)', size=(40,1)), sg.Input(size=(30,1), key='Codigo_Nat'), sg.FileBrowse('Buscar')],
    [sg.Button('OK'), sg.Exit('Cancelar')]
]


window = sg.Window('Organizador de Archivos', layout)

while True:             # Event Loop
    event, values = window.Read()
    if event in (None, 'Cancelar'):
        break
    elif event == 'OK':
        organizador(values)
        
window.Close()


